# Notebook 3: Model Training and Evaluation

This notebook trains the three ML model architectures and evaluates them
using leave-one-isotope-out cross-validation.

Models trained:
1. XGBoost with quantile regression for uncertainty
2. Random Forest with ensemble variance uncertainty
3. Neural Network with MC dropout uncertainty
4. Ensemble combining all three

Evaluation strategy: leave-one-isotope-out CV, where all data for
each isotope is held out in turn to simulate prediction for unmeasured nuclei.

In [ ]:
import sys
sys.path.insert(0, '../src')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

from data.dataset_builder import DatasetBuilder, FEATURE_COLUMNS
from models.xgboost_model import XGBoostCrossSectionModel
from models.random_forest_model import RandomForestCrossSectionModel
from models.neural_network import NeuralNetworkCrossSectionModel
from models.ensemble import EnsembleCrossSectionModel
from evaluation.metrics import (
    compute_metrics, mass_region_breakdown,
    compute_factor_of_x_accuracy, calibration_error
)
from evaluation.cross_validation import run_loio_cv
from visualization.plots import (
    plot_predicted_vs_experimental,
    plot_feature_importance,
    plot_uncertainty_calibration,
    plot_mass_region_accuracy,
)

plt.rcParams.update({'font.family': 'serif', 'figure.dpi': 150})
print('Setup complete.')

## 1. Load dataset

In [ ]:
dataset_path = '../data/processed/n_gamma_dataset.h5'

if Path(dataset_path).exists():
    splits = DatasetBuilder.load_from_hdf5(dataset_path)
    X_train, y_train = splits['train']
    X_val, y_val = splits['val']
    X_test, y_test = splits['test']
    print(f'Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}')
else:
    print(f'Dataset not found. Generating synthetic data for demonstration...')
    rng = np.random.default_rng(42)
    n_train, n_val, n_test = 3000, 500, 500
    n_features = len(FEATURE_COLUMNS)
    X_train = rng.standard_normal((n_train, n_features)).astype(np.float32)
    X_val = rng.standard_normal((n_val, n_features)).astype(np.float32)
    X_test = rng.standard_normal((n_test, n_features)).astype(np.float32)
    # Target: simple function for demonstration
    y_train = (2 * X_train[:, 0] - X_train[:, 2] + 0.5 * rng.standard_normal(n_train)).astype(np.float32)
    y_val = (2 * X_val[:, 0] - X_val[:, 2] + 0.5 * rng.standard_normal(n_val)).astype(np.float32)
    y_test = (2 * X_test[:, 0] - X_test[:, 2] + 0.5 * rng.standard_normal(n_test)).astype(np.float32)
    print(f'Synthetic: Train={n_train}, Val={n_val}, Test={n_test}, Features={n_features}')

## 2. Train XGBoost

In [ ]:
xgb_model = XGBoostCrossSectionModel(
    params={'n_estimators': 500, 'max_depth': 6, 'learning_rate': 0.05,
            'subsample': 0.8, 'colsample_bytree': 0.8, 'min_child_weight': 3},
    feature_names=FEATURE_COLUMNS,
    early_stopping_rounds=30,
)
xgb_model.fit(X_train, y_train, X_val, y_val)

xgb_metrics = xgb_model.evaluate(X_test, y_test)
print('XGBoost test metrics:')
for k, v in sorted(xgb_metrics.items()):
    print(f'  {k}: {v:.4f}')

## 3. Train Random Forest

In [ ]:
rf_model = RandomForestCrossSectionModel(
    params={'n_estimators': 300, 'max_depth': 15, 'min_samples_leaf': 5,
            'max_features': 'sqrt', 'n_jobs': -1, 'random_state': 42},
    feature_names=FEATURE_COLUMNS,
)
rf_model.fit(X_train, y_train, X_val, y_val)

rf_metrics = rf_model.evaluate(X_test, y_test)
print('Random Forest test metrics:')
for k, v in sorted(rf_metrics.items()):
    print(f'  {k}: {v:.4f}')

## 4. Train Neural Network

In [ ]:
nn_model = NeuralNetworkCrossSectionModel(
    n_features=X_train.shape[1],
    hidden_dims=(128, 128, 64),
    dropout_rate=0.2,
    learning_rate=1e-3,
    batch_size=256,
    max_epochs=100,
    patience=15,
    mc_samples=50,
)
nn_model.fit(X_train, y_train, X_val, y_val)

nn_metrics = nn_model.evaluate(X_test, y_test)
print('Neural Network test metrics:')
for k, v in sorted(nn_metrics.items()):
    print(f'  {k}: {v:.4f}')

## 5. Model comparison

In [ ]:
# Build ensemble
ensemble = EnsembleCrossSectionModel(
    models=[xgb_model, rf_model, nn_model],
    model_names=['XGBoost', 'RandomForest', 'NeuralNetwork'],
)
ensemble.fit_meta_learner(X_val, y_val)
ens_metrics = ensemble.evaluate(X_test, y_test)

# Summary table
all_metrics = {
    'XGBoost': xgb_metrics,
    'RandomForest': rf_metrics,
    'NeuralNetwork': nn_metrics,
    'Ensemble': ens_metrics,
}

summary_rows = []
for model_name, m in all_metrics.items():
    summary_rows.append({
        'Model': model_name,
        'RMSE': m.get('rmse', float('nan')),
        'MAE': m.get('mae', float('nan')),
        'R2': m.get('r2', float('nan')),
        'PICP_68%': m.get('picp_68', float('nan')),
        'MPIW_68%': m.get('mpiw_68', float('nan')),
    })

summary_df = pd.DataFrame(summary_rows).set_index('Model')
print('Model comparison on test set:')
display(summary_df.style.highlight_min(subset=['RMSE', 'MAE'], color='lightgreen')
                        .highlight_max(subset=['R2'], color='lightgreen')
                        .format('{:.4f}'))

## 6. Predicted vs experimental plots

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 12))

model_list = [
    ('XGBoost', xgb_model),
    ('Random Forest', rf_model),
    ('Neural Network', nn_model),
    ('Ensemble', ensemble),
]

for ax, (name, model) in zip(axes.flat, model_list):
    preds = model.predict_with_uncertainty(X_test)
    pred_key = 'mean' if 'mean' in preds else 'median'
    y_pred = preds[pred_key]
    
    lim_min = min(y_test.min(), y_pred.min()) - 0.3
    lim_max = max(y_test.max(), y_pred.max()) + 0.3
    x_line = np.linspace(lim_min, lim_max, 100)
    
    ax.fill_between(x_line, x_line - np.log10(2), x_line + np.log10(2),
                    alpha=0.15, color='steelblue')
    ax.plot(x_line, x_line, 'k-', lw=1)
    ax.scatter(y_test, y_pred, s=5, alpha=0.4, color='navy', rasterized=True)
    
    rmse = np.sqrt(np.mean((y_pred - y_test)**2))
    r2 = 1 - np.sum((y_test - y_pred)**2) / np.sum((y_test - y_test.mean())**2)
    ax.text(0.97, 0.05, f'RMSE={rmse:.3f}\nR²={r2:.3f}',
            transform=ax.transAxes, ha='right', va='bottom',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    ax.set_xlim(lim_min, lim_max)
    ax.set_ylim(lim_min, lim_max)
    ax.set_xlabel('Experimental log$_{10}$($\\sigma$ / barn)')
    ax.set_ylabel('Predicted log$_{10}$($\\sigma$ / barn)')
    ax.set_title(name)
    ax.set_aspect('equal')

plt.suptitle('Predicted vs experimental (n,$\\gamma$) cross-sections', y=1.01)
plt.tight_layout()
plt.savefig('../results/figures/all_models_pred_vs_exp.pdf', dpi=300, bbox_inches='tight')
plt.show()

## 7. Feature importance (XGBoost)

In [ ]:
try:
    imp_df = xgb_model.feature_importance()
    fig = plot_feature_importance(
        imp_df, top_n=15,
        title='XGBoost Feature Importance (Information Gain)',
        output_path='../results/figures/xgboost_feature_importance.pdf'
    )
    plt.show()
    print('Top 5 most important features:')
    display(imp_df.head())
except Exception as e:
    print(f'Feature importance not available: {e}')

## 8. Uncertainty calibration

In [ ]:
# Calibration plots for all models with uncertainty
calibration_results = {}

for name, model in [('XGBoost', xgb_model), ('RandomForest', rf_model), ('NeuralNetwork', nn_model)]:
    preds = model.predict_with_uncertainty(X_test)
    pred_key = 'mean' if 'mean' in preds else 'median'
    std_key = 'std' if 'std' in preds else ('sigma_68' if 'sigma_68' in preds else None)
    
    if std_key:
        cal = calibration_error(y_test, preds[pred_key], preds[std_key])
        calibration_results[name] = cal
        print(f'{name}: ECE = {cal["ece"]:.4f}')

if calibration_results:
    expected_list = [r['expected_coverage'] for r in calibration_results.values()]
    observed_list = [r['observed_coverage'] for r in calibration_results.values()]
    labels = list(calibration_results.keys())
    
    fig = plot_uncertainty_calibration(
        expected_list, observed_list,
        model_labels=labels,
        output_path='../results/figures/uncertainty_calibration.pdf'
    )
    plt.show()
else:
    print('Uncertainty estimates not available for calibration plot.')